## Мультиклассовая классификация

In [25]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики

import warnings # для игнорирования предупреждений
#Игнорируем варнинги
warnings.filterwarnings('ignore')


# Устанавливаем стиль визуализаций в matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8')

In [26]:
# загрузка дата-сета
penguins = pd.read_csv('data/penguins1.csv')
df = penguins.copy()
df.head()
# присутствуют пропущенные значения

,Unnamed: 0,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [27]:
#Удалим первый столбец
df.drop(columns=df.columns [0], axis=1, inplace=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


Описание данных:

* species — класс пингвина ('Adelie', 'Chinstrap', 'Gentoo'), целевой признак;
* island — остров, на котором живёт пингвин ('Torgersen', 'Biscoe', 'Dream');
* bill_length_mm — длина клюва в миллиметрах;
* bill_depth_mm — толщина клюва в миллиметрах;
* flipper_length_mm — длина крыльев;
* body_mass_g — масса;
* sex — пол ('Male', 'Female').

Наша цель — предсказать класс пингвина.

Очевидно, что классификация является мультиклассовой, так как возможных значений три.

Для начала давайте посмотрим на пропуски в данных:

In [29]:
# анализируем пропуски
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [30]:
# Логистическая регрессия не работает с пропусками, удалим пропуски
df = df.dropna()
df.isnull().sum()
# Пропуски удалены

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

Разделим дата-сет на матрицу наблюдений - `X` и столбец с правильными ответами `y`

In [31]:
X = df.drop('species', axis=1)
y = df['species']

Кодировать целевую переменную "y" не нужно, даже если она представлена в виде строкового значения. Это не влияет на работу модели.

In [39]:
# анализируем уникальные значения в каждом категориальном признаке
# формируем список категориальных признаков
obj = []
for i in X.columns:
    if X[i].dtypes  == 'object':
        obj.append(i)
print(f'Столбцы "object":\n {obj}')
# два категориальных признака

Столбцы "object":
 ['island', 'sex']


In [37]:
# возвращаем кол-во уникальных значений в категориальных признаках
for col in obj:
    print(col, X[col].unique())
# уникальных признаков до 3-х видов, расширимся на 5 столбцов, можно использовать горячее кодирование get_dummies

island ['Torgersen' 'Biscoe' 'Dream']
sex ['Male' 'Female']


In [38]:
# кодируем методом get_dummies()
X_dummies = pd.get_dummies(X)
X_dummies.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
0,39.1,18.7,181.0,3750.0,0,0,1,0,1
1,39.5,17.4,186.0,3800.0,0,0,1,1,0
2,40.3,18.0,195.0,3250.0,0,0,1,1,0
4,36.7,19.3,193.0,3450.0,0,0,1,1,0
5,39.3,20.6,190.0,3650.0,0,0,1,0,1
